Get spacy and gensim:

``conda install -c conda-forge spacy``\
``python -m spacy download en_core_web_lg``\
``conda install -c conda-forge gensim``


In [1]:
import pandas as pd

df = pd.read_csv('mbti_1.csv')
# split_posts = df['posts'].to_string().split('|||')

In [2]:
import numpy as np

# break up each row into its constituent sentences
# drop any row that has more than 50 sentences (these often have ||||||| in them)
l = df['posts'].values.tolist()
targets = df['type'].values.tolist()
new_list_1 = [x.split('|||') for x in l]

for i in range(len(new_list_1)):
    new_list_1[i].insert(0, targets[i])
    
new_list = [x for x in new_list_1 if len(x) <= 51]
df_mbti = pd.DataFrame(new_list)


In [7]:
from sklearn.model_selection import train_test_split

text_X_train, text_X_test, text_y_train, text_y_test = train_test_split(df_mbti.drop(columns=[0]),
                                                    df_mbti[0],
                                                    test_size = 0.2,
                                                    random_state = 123)

# reshape train data so each sentence is on its own row, remove Nones
text_X_train = text_X_train.to_numpy().reshape(-1,1)
text_y_train = np.c_[[text_y_train for i in range(50)]].T.reshape(-1,1)
text_train_data = np.c_[text_y_train, text_X_train]
# if someone gave less than 50 responses, drop the last 'none' text
text_train_data = text_train_data[text_train_data[:,1] != None]

text_X_test = text_X_test.to_numpy().reshape(-1,1)
text_y_test = np.c_[[text_y_test for i in range(50)]].T.reshape(-1,1)
text_test_data = np.c_[text_y_test, text_X_test]
text_test_data = text_test_data[text_test_data[:,1] != None]


print(text_test_data)


[['ISFP'
  "Now that you mention it, I see a lot of Christian Grey style ENTJ-ness in Narcisse (I'm pretty sure Anastasia was also an INFJ). Their overall relationship quite reminds me of Fifty Shades, which is..."]
 ['ISFP'
  "I'm a high school senior so don't judge me! I get to school like an hour early and I head to the art room but it's locked so I knock on the office door to see if the teacher's inside. My teacher..."]
 ['ISFP'
  "Definitely ENTP! Don't over think this. It takes one to know one."]
 ...
 ['INTP'
  "Not yet. I could do it. But I most match with Ne and Se what just can't work. .-."]
 ['INTP' "I've read very much about it. xD"]
 ['INTP'
  "At first I thoght I'm an ISTJ, but I'm not sure anymore. But there is so much N. I'm very creative, esspecially when writing music. I have soo many ideas for covers when I'm listening to music. I...'"]]


In [35]:
# convert labels to numbers

labels16 = [0 for _ in range(text_test_data.shape[0])]
labels4 = [[0, 0, 0, 0] for _ in range(text_test_data.shape[0])]
for i in range(len(labels16)):
    if text_test_data[i,0][0] == "I": #E
        labels16[i] += 8
        labels4[i][0] = 1
    if text_test_data[i,0][1] == "S": #N
        labels16[i] += 4
        labels4[i][1] = 1
    if text_test_data[i,0][2] == "T": #F
        labels16[i] += 2
        labels4[i][2] = 1
    if text_test_data[i,0][3] == "P": #J
        labels16[i] += 1
        labels4[i][3] = 1
    
labels16 = np.array(labels16)
labels4 = np.array(labels4)

In [40]:
import spacy
nlp_model = spacy.load('en_core_web_lg') # 300 features

# size of embedding
num_features = 10

train_data = np.zeros((text_train_data.shape[0], num_features))

for i, line in enumerate(text_train_data[:10,]):
    doc = nlp_model(line[1])
    train_data[i,:] = doc.vector[:num_features]
print(train_data)
print(train_data.shape)



[[-1.27890691e-01  2.95791060e-01 -1.13752224e-01 ... -1.66094676e-01
  -7.53668323e-02  2.02152467e+00]
 [-1.29937604e-02  1.43967971e-01 -2.23346069e-01 ... -1.51102468e-01
   1.64454244e-02  2.12484646e+00]
 [ 2.65318099e-02  2.26032540e-01 -1.82158291e-01 ... -2.25909084e-01
  -1.24365091e-04  1.79578197e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
(333202, 10)


In [43]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]

model = Doc2Vec(documents, vector_size=num_features, window=2, min_count=1, workers=4)

vector2 = model.infer_vector(["'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the un..."])

print(vector2)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [42]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]

num_features = 300
model = Doc2Vec(documents, vector_size=num_features, window=2, min_count=1, workers=4)

from gensim.test.utils import get_tmpfile


fname = get_tmpfile("my_doc2vec_model")


model.save(fname)

model = Doc2Vec.load(fname)  # you can continue training with the loaded model!


vector2 = model.infer_vector(["'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the un..."])
vector3 = model.infer_vector(["'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; t..."])
vector4 = model.infer_vector(["'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same po..."])


print(vector4.shape)
print(vector3)
print(vector2)


ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [8]:

import spacy

# load language model
# nlp_model = spacy.load('en_core_web_sm') # 96 features
nlp_model = spacy.load('en_core_web_lg') # 300 features

text = "knock knocked bring brought"

doc = nlp_model(text)

print(doc[0].vector[:10])
print(doc.vector[:10])

from scipy.spatial.distance import cosine
knock = doc[0].vector
knocked = doc[1].vector
bring = doc[2].vector
brought = doc[3].vector

print()
print(knock.shape)

print(brought @ brought.T)
brought_2 = knocked - knock + bring
print(brought_2 @ brought_2.T)
print(brought_2 @ brought.T)


print(cosine(brought, brought))
print(cosine(brought_2, brought))
print(cosine(bring, brought))
print(cosine(knocked, brought))
print(cosine(knock, knocked))



[-0.48385    0.067619  -0.11923    0.29677   -0.0024814 -0.43181
 -0.15166    0.10901    0.35108    1.7757   ]
[-0.23949926 -0.00695597 -0.06731676  0.24106626 -0.01212585 -0.3377375
 -0.11993999  0.08199837 -0.0297675   2.4574    ]

(300,)
21.791866
30.923595
19.217073
0.0
0.2597215175628662
0.2976776957511902
0.5519660711288452
0.24341750144958496
